This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](end-conditions.ipynb)

# Natural End Conditions

For the first and last segment, we assume that the inner tangent is known.
To find the outer tangent
according to *natural* end conditions,
the second derivative is set to $0$
at the beginning and end of the curve.

We are looking only at the non-uniform case here,
it's easy to get to the uniform case by setting $\Delta_i = 1$.

Natural end conditions are naturally a good fit for
[natural splines](natural-uniform.ipynb#End-Conditions).
And in case you were wondering,
natural end conditions are sometimes also called "relaxed" end conditions.

In [ ]:
import sympy as sp
sp.init_printing(order='grevlex')

As usual, we are getting some help from [utility.py](utility.py):

In [ ]:
from utility import NamedExpression

In [ ]:
t = sp.symbols('t')

## Begin

We are starting with the first polynomial segment $\boldsymbol{p}_0(t)$,
with $t_0 \le t \le t_1$.

In [ ]:
t0, t1 = sp.symbols('t:2')

The coefficients ...

In [ ]:
a0, b0, c0, d0 = sp.symbols('a:dbm0')

... multiplied with the [monomial basis](polynomials.ipynb)
give us the uniform polynomial ...

In [ ]:
d0 * t**3 + c0 * t**2 + b0 * t + a0

... which we re-scale to the desired parameter range:

In [ ]:
p0 = NamedExpression('pbm0', _.subs(t, (t - t0) / (t1 - t0)))
p0

We need the first derivative
(a.k.a velocity, a.k.a. tangent vector):

In [ ]:
pd0 = p0.diff(t)
pd0

Similar to the
[notebook about non-uniform Hermite splines](hermite-non-uniform.ipynb),
we are interested in the function values and first derivatives
at the control points:

\begin{align*}
\boldsymbol{x}_0 &= \boldsymbol{p}_0(t_0)\\
\boldsymbol{x}_1 &= \boldsymbol{p}_0(t_1)\\
\boldsymbol{\dot{x}}_0 &= \boldsymbol{p}_0'(t_0)\\
\boldsymbol{\dot{x}}_1 &= \boldsymbol{p}_0'(t_1)
\end{align*}

In [ ]:
equations_begin = [
    p0.evaluated_at(t, t0).with_name('xbm0'),
    p0.evaluated_at(t, t1).with_name('xbm1'),
    pd0.evaluated_at(t, t0).with_name('xdotbm0'),
    pd0.evaluated_at(t, t1).with_name('xdotbm1'),
]

To get simpler equations,
we are substituting $\Delta_0 = t_1 - t_0$.
Note that this is only for display purposes,
the calculations are still done with $t_i$.

In [ ]:
delta_begin = [
    (t0, 0),
    (t1, sp.Symbol('Delta0')),
]

In [ ]:
for e in equations_begin:
    display(e.subs(delta_begin))

In [ ]:
coefficients_begin = sp.solve(equations_begin, [a0, b0, c0, d0])

In [ ]:
for c, e in coefficients_begin.items():
    display(NamedExpression(c, e.subs(delta_begin)))

The second derivative (a.k.a. acceleration) ...

In [ ]:
pdd0 = pd0.diff(t)
pdd0

... at the beginning of the curve ($t = t_0$) ...

In [ ]:
pdd0.evaluated_at(t, t0)

... is set to zero ...

In [ ]:
sp.Eq(_.expr, 0).subs(coefficients_begin)

... leading to an expression for the initial tangent vector:

In [ ]:
xd0 = NamedExpression.solve(_, 'xdotbm0')
xd0.subs(delta_begin)

This can also be written as

\begin{equation*}
\boldsymbol{\dot{x}}_0 =
\frac{3 \left(\boldsymbol{x}_1 - \boldsymbol{x}_0\right)}{2 \Delta_0} -
\frac{\boldsymbol{\dot{x}}_1}{2}.
\end{equation*}

## End

If a spline has $N$ vertices,
it has $N-1$ polynomial segments
and the last polynomial segment is
$\boldsymbol{p}_{N-2}(t)$, with $t_{N-2} \le t \le t_{N-1}$.
To simplify the notation a bit,
let's assume we have $N = 10$ vertices,
which makes $\boldsymbol{p}_8$ the last polynomial segment.
The following steps are very similar
to the above derivation of the start conditions.

In [ ]:
a8, b8, c8, d8 = sp.symbols('a:dbm8')

In [ ]:
t8, t9 = sp.symbols('t8:10')

In [ ]:
d8 * t**3 + c8 * t**2 + b8 * t + a8

In [ ]:
p8 = NamedExpression('pbm8', _.subs(t, (t - t8) / (t9 - t8)))
p8

In [ ]:
pd8 = p8.diff(t)
pd8

\begin{align*}
\boldsymbol{x}_{N-2} &= \boldsymbol{p}_{N-2}(t_{N-2})\\
\boldsymbol{x}_{N-1} &= \boldsymbol{p}_{N-2}(t_{N-1})\\
\boldsymbol{\dot{x}}_{N-2} &= \boldsymbol{p}_{N-2}'(t_{N-2})\\
\boldsymbol{\dot{x}}_{N-1} &= \boldsymbol{p}_{N-2}'(t_{N-1})
\end{align*}

In [ ]:
equations_end = [
    p8.evaluated_at(t, t8).with_name('xbm8'),
    p8.evaluated_at(t, t9).with_name('xbm9'),
    pd8.evaluated_at(t, t8).with_name('xdotbm8'),
    pd8.evaluated_at(t, t9).with_name('xdotbm9'),
]

We define $\Delta_8 = t_9 - t_8$:

In [ ]:
delta_end = [
    (t8, 0),
    (t9, sp.Symbol('Delta8')),
]

In [ ]:
for e in equations_end:
    display(e.subs(delta_end))

In [ ]:
coefficients_end = sp.solve(equations_end, [a8, b8, c8, d8])

In [ ]:
for c, e in coefficients_end.items():
    display(NamedExpression(c, e.subs(delta_end)))

This time,
the second derivative ...

In [ ]:
pdd8 = pd8.diff(t)
pdd8

... *at the end* of the last segment ($t = t_9$) ...

In [ ]:
pdd8.evaluated_at(t, t9)

... is set to zero ...

In [ ]:
sp.Eq(_.expr, 0).subs(coefficients_end)

... leading to an expression for the final tangent vector:

In [ ]:
xd9 = NamedExpression.solve(_, 'xdotbm9')
xd9.subs(delta_end)

Luckily, that's symmetric to the result we got above.

The equation can be generalized to

\begin{equation*}
\boldsymbol{\dot{x}}_{N-1} =
\frac{3 \left(\boldsymbol{x}_{N-1} - \boldsymbol{x}_{N-2}\right)}{2 \Delta_{N-2}} -
\frac{\boldsymbol{\dot{x}}_{N-2}}{2}.
\end{equation*}

## Example

We are showing a one-dimensional example where 3 time/value pairs are given.
The slope for the middle value is given, the begin and end slopes are calculated using the "natural" end conditions as calculated above.

In [ ]:
values = 2, 2, 1
times = 0, 4, 5
slope = 2

We are using a few helper functions from [helper.py](helper.py) for plotting:

In [ ]:
from helper import plot_sympy, grid_lines

In [ ]:
x0, x1 = sp.symbols('xbm0:2')
x8, x9 = sp.symbols('xbm8:10')
xd1 = sp.symbols('xdotbm1')
xd8 = sp.symbols('xdotbm8')

In [ ]:
begin = p0.subs(coefficients_begin).subs_symbols(xd0).subs({
    t0: times[0],
    t1: times[1],
    x0: values[0],
    x1: values[1],
    xd1: slope,
}).with_name(r'p_\text{begin}')
end = p8.subs(coefficients_end).subs_symbols(xd9).subs({
    t8: times[1],
    t9: times[2],
    x8: values[1],
    x9: values[2],
    xd8: slope,
}).with_name(r'p_\text{end}')

In [ ]:
plot_sympy(
    (begin.expr, (t, times[0], times[1])),
    (end.expr, (t, times[1], times[2])))
grid_lines(times, [1, 2])

In [ ]:
begin.diff(t).evaluated_at(t, times[0])

In [ ]:
end.diff(t).evaluated_at(t, times[-1])

## Bézier Control Points

Up to now we have assumed
that we know one of the tangent vectors
and want to find the other tangent vector
in order to construct a [Hermite spline](hermite.ipynb).
What if we want to construct a [Bézier spline](bezier.ipynb) instead?

If the inner Bézier control points
$\boldsymbol{\tilde{x}}_1^{(-)}$ and
$\boldsymbol{\tilde{x}}_{N-2}^{(+)}$
are given,
we can insert the equations for the tangent vectors from the
[notebook about non-uniform Bézier splines](bezier-non-uniform.ipynb#Tangent-Vectors)
into our tangent vector equations from above
and solve them for the outer control points
$\boldsymbol{\tilde{x}}_0^{(+)}$ and
$\boldsymbol{\tilde{x}}_{N-1}^{(-)}$, respectively.

In [ ]:
xtilde0, xtilde1 = sp.symbols('xtildebm0^(+) xtildebm1^(-)')

In [ ]:
NamedExpression.solve(xd0.subs({
    xd0.name: 3 * (xtilde0 - x0) / (t1 - t0),
    xd1: 3 * (x1 - xtilde1) / (t1 - t0),
}), xtilde0)

In [ ]:
xtilde8, xtilde9 = sp.symbols('xtildebm8^(+) xtildebm9^(-)')

In [ ]:
NamedExpression.solve(xd9.subs({
    xd8: 3 * (xtilde8 - x8) / (t9 - t8),
    xd9.name: 3 * (x9 - xtilde9) / (t9 - t8),
}), xtilde9)

Note that all $\Delta_i$ cancel each other out
(as well as the inner vertices
$\boldsymbol{x}_1$ and
$\boldsymbol{x}_{N-2}$)
and we get very simple equations for the "natural" end conditions:

\begin{align*}
\boldsymbol{\tilde{x}}_0^{(+)} &=
\frac{\boldsymbol{x}_0 + \boldsymbol{\tilde{x}}_1^{(-)}}{2} \\
\boldsymbol{\tilde{x}}_{N-1}^{(-)} &=
\frac{\boldsymbol{x}_{N-1} + \boldsymbol{\tilde{x}}_{N-2}^{(+)}}{2}
\end{align*}